# Web Crawling con Python

El objetivo de este trabajo práctico es implementar un web crawler en Python que pueda
recorrer un sitio web, extraer todas las etiquetas `<a>` con sus respectivos enlaces y acceder a
cada página enlazada. Por cada enlace encontrado, se deben obtener todas las etiquetas `<h1>` y
`<p>` y almacenarlo en un array en un archivo JSON y si no se encuentran dichos elementos
guardar el array como vacío. Por ejemplo, si el crawler encuentra un enlace
https://example.com/pagina1 en una página y accede a ese enlace, debe obtener el contenido
de la página https://example.com/pagina1 y buscar todos los elementos `<h1></h1>` y
`<p></p>`, luego almacenarlo en un array en un archivo JSON bajo la clave
"https://example.com/pagina1" con el array de los elementos solicitados de la página como
valor correspondiente y si no se encuentran dichos elementos guardar el array vacío.



1. Importamos las librerias 

In [ ]:
import requests
from bs4 import BeautifulSoup
import json

2. Creamos una funcion para extraer los datos de una pagina web

In [ ]:
def rastrear_sitio_web(url): 
    try:
        respuesta = requests.get(url) # Realiza una solicitud GET a la URL proporcionada
        respuesta.raise_for_status()  # Verifica si hay errores HTTP en la respuesta
        soup = BeautifulSoup(respuesta.text, 'html.parser')  # Crea un objeto BeautifulSoup para analizar el contenido HTML
        
        enlace = soup.find_all('a', href=True)[:100]  # Encuentra todas las etiquetas <a> con atributo href y obtiene los primeros 100 enlaces
                                                      # Utilice [:100] para limitar la búsqueda a los primeros 100 enlaces para evitar sobrecarga de recursos
                                                      # Para obtener todos los enlaces, elimine [:100] o comentelo

        datos = {}  # Inicializa un diccionario para almacenar los datos extraídos de las páginas enlazadas
        
        for enlace in enlace:  # Itera sobre cada enlace encontrado
            href = enlace['href']  # Obtiene la URL del enlace
            if href.startswith('http'):  # Verifica si el enlace comienza con 'http', es decir, si es un enlace externo.
                try:
                    respuesta_pagina = requests.get(href)  # Realiza una solicitud GET al enlace
                    respuesta_pagina.raise_for_status()  # Verifica si hay errores HTTP en la respuesta
                    pagina_soup = BeautifulSoup(respuesta_pagina.text, 'html.parser')  # Crea un objeto BeautifulSoup para analizar el contenido HTML de la página enlazada
                    etiquetas_h1 = pagina_soup.find_all('h1')  # Encuentra todas las etiquetas <h1> en la página enlazada
                    etiquetas_p = pagina_soup.find_all('p')  # Encuentra todas las etiquetas <p> en la página enlazada
                    if etiquetas_h1 or etiquetas_p:  # Verifica si hay etiquetas <h1> o <p> en la página enlazada
                        datos[href] = [str(tag) for tag in etiquetas_h1 + etiquetas_p]  # Almacena las etiquetas encontradas como una lista de cadenas en el diccionario 'datos'
                    else:
                        datos[href] = []  # Si no se encuentran etiquetas <h1> ni <p>, almacena una lista vacía en el diccionario 'datos'
                except requests.exceptions.RequestException as e:  # Maneja las excepciones relacionadas con las solicitudes HTTP
                    print(f"Error al acceder a {href}: {e}") 
                except Exception as e:  # Maneja cualquier otro tipo de excepción
                    print(f"Error inesperado al procesar {href}: {e}") 
                    
        return datos  # Retorna el diccionario 'datos' con los datos extraídos de las páginas enlazadas
    
    except requests.exceptions.RequestException as e:  # Maneja las excepciones relacionadas con las solicitudes HTTP
        print("Error al realizar la solicitud:", e)  # Imprime un mensaje de error
        return {}  # Retorna un diccionario vacío si ocurre un error en la solicitud HTTP

3. Creamos otra funcion en donde guardamos los datos extraidos en un archivo JSON.

In [ ]:
# Definimos una función que toma dos argumentos: `datos` (los datos a guardar) y `nombre_archivo` (el nombre del archivo).
def guardar_en_json(datos, nombre_archivo):
    
    with open(nombre_archivo, 'w') as archivo:
        # Abre el archivo especificado por `nombre_archivo` en modo escritura ('w').
        # Utilizamos `with` para garantizar que el archivo se cierre correctamente. 
        
        json.dump(datos, archivo, indent=4)
        # Convierte los datos proporcionados (`datos`) a formato JSON y los escribe en el archivo.
        # `indent=4` agrega una indentación de 4 espacios para hacer el archivo más legible.


4. Luego verificamos si es el principal, definimos una URL y un archivo de salida, rastreamos el sitio web y guardamos los datos en un archivo JSON.

In [ ]:
if __name__ == "__main__":
    # Verifica si este script está siendo ejecutado como el programa principal.
    
    url = 'https://es.wikipedia.org/wiki/Wikipedia:Portada'
    # Define la URL del sitio web que se va a rastrear.
    
    nombre_archivo_salida = 'datos_sitio_web.json'
    # Define el nombre del archivo donde se guardarán los datos del sitio web.
    
    datos_sitio_web = rastrear_sitio_web(url)
    # Llama a la función `rastrear_sitio_web` con la URL especificada para obtener los datos del sitio web.
    
    guardar_en_json(datos_sitio_web, nombre_archivo_salida)
    # Guarda los datos del sitio web en un archivo JSON utilizando la función `guardar_en_json`.
    # `datos_sitio_web` contiene los datos del sitio web obtenidos de `rastrear_sitio_web`.
    # `nombre_archivo_salida` es el nombre del archivo donde se guardarán los datos.
